In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
sns.set()

In [2]:
GSPREADHSEET_DOWNLOAD_URL = (
    "https://docs.google.com/spreadsheets/d/{gid}/export?format=csv&id={gid}".format
)

FIUFIP_2021_1_GID = '1-DWTP8uwVS-dZY402-dm0F9ICw_6PNqDGLmH0u8Eqa0'
df = pd.read_csv(GSPREADHSEET_DOWNLOAD_URL(gid=FIUFIP_2021_1_GID))

In [3]:
df.head()

,anios_estudiados,barrio,categoria_de_trabajo,edad,educacion_alcanzada,estado_marital,ganancia_perdida_declarada_bolsa_argentina,genero,horas_trabajo_registradas,religion,rol_familiar_registrado,tiene_alto_valor_adquisitivo,trabajo
0,17,Palermo,empleado_provincial,39,universidad_3_anio,sin_matrimonio,2174,hombre,40,cristianismo,sin_familia,0,entretenimiento
1,17,Palermo,monotibutista,50,universidad_3_anio,matrimonio_civil,0,hombre,13,cristianismo,casado,0,directivo_gerente
2,13,Palermo,relacion_de_dependencia,38,4_anio,divorciado,0,hombre,40,cristianismo,sin_familia,0,limpiador
3,11,Palermo,relacion_de_dependencia,53,2_anio,matrimonio_civil,0,hombre,40,judaismo,casado,0,limpiador
4,17,Balvanera,relacion_de_dependencia,28,universidad_3_anio,matrimonio_civil,0,mujer,40,judaismo,casada,0,profesional_especializado


# Feature Engineering

## Missings

### Hay valores inválidos?

In [4]:
print(df.astype('str').eq('-').any(0).value_counts())
print(df.astype('str').eq('').any(0).value_counts())
(df[['edad', 'horas_trabajo_registradas']] < 0).any()

False    13
dtype: int64
False    13
dtype: int64


edad                         False
horas_trabajo_registradas    False
dtype: bool

### ¿Hay missings?

In [5]:
print((df.isnull().mean() * 100).to_frame('Porcentaje de nulos'))

                                            Porcentaje de nulos
anios_estudiados                                       0.000000
barrio                                                 1.790486
categoria_de_trabajo                                   5.638647
edad                                                   0.000000
educacion_alcanzada                                    0.000000
estado_marital                                         0.000000
ganancia_perdida_declarada_bolsa_argentina             0.000000
genero                                                 0.000000
horas_trabajo_registradas                              0.000000
religion                                               0.000000
rol_familiar_registrado                                0.000000
tiene_alto_valor_adquisitivo                           0.000000
trabajo                                                5.660146


- Se idenfican los features con missings y se observa que poseen porcentajes bajos.

### ¿Como tratar los missings presentes?

- Primero, se analizan las categorías presentes para entender si existe una categoría asociada a respuestas no informadas.

In [6]:
print(*(df['barrio'].dropna().unique()), sep= "\n") 

Palermo
Balvanera
Caballito
Almagro
Belgrano
Boedo
La Boca
Versalles
Puerto Madero
Recoleta
Villa Urquiza
Nueva Pompeya
San Isidro
Chacarita
Flores
Liniers
Villa Devoto
San Telmo
Parque Chas
Villa Real
Monte Castro
Mataderos
Monserrat
Coghland
Agronomia
Villa Luro
Constitucion
Barracas
Colegiales
Santa Rita
Parque Chacabuco
Villa General Mitre
Villa Pueyrredon
Villa Soldati
Parque Avellaneda
nuñez
Floresta
Retiro
La Paternal
Velez Sarsfield
Cilla Riachuelo


In [7]:
print(*(df['categoria_de_trabajo'].dropna().unique()), sep = "\n")

empleado_provincial
monotibutista
relacion_de_dependencia
empleadao_estatal
empleado_municipal
responsable_inscripto
trabajo_voluntariado
sin_trabajo


In [8]:
print(*(df['trabajo'].dropna().unique()), sep="\n")

entretenimiento
directivo_gerente
limpiador
profesional_especializado
otros
ventas
reparador
transporte
sector_primario
inspector
soporte_tecnico
seguridad
ejercito
servicio_domestico


- Clasificación de missings por feature: 
    - barrio: Completamente aleatorios
    - categoria_de_trabajo: Completamente aleatorios
    - trabajo: Aleatorios
- Tratamiento de missings: para barrio y categoria_de_trabajo se asignarán nuevas categorías, en ambos casos llamadas "sin_informar". Para trabajo se hará lo mismo, pero en los casos en que categoria_de_trabajo sea sin_trabajo se asignará una nueva categoria llamada "sin_trabajo". 

In [9]:
df['barrio'] = df['barrio'].replace(np.nan, 'sin_informar')
df['categoria_de_trabajo'] = df['categoria_de_trabajo'].replace(np.nan, 'sin_informar')
df['trabajo'] = df['trabajo'].replace(np.nan, 'sin_informar')
df.loc[df['categoria_de_trabajo'] == 'sin_trabajo', ['trabajo']] = 'sin_trabajo'

In [10]:
print((df.isnull().mean() * 100).to_frame('Porcentaje de nulos'))

                                            Porcentaje de nulos
anios_estudiados                                            0.0
barrio                                                      0.0
categoria_de_trabajo                                        0.0
edad                                                        0.0
educacion_alcanzada                                         0.0
estado_marital                                              0.0
ganancia_perdida_declarada_bolsa_argentina                  0.0
genero                                                      0.0
horas_trabajo_registradas                                   0.0
religion                                                    0.0
rol_familiar_registrado                                     0.0
tiene_alto_valor_adquisitivo                                0.0
trabajo                                                     0.0


- Se observa que todos los missings expuestos han sido neutralizados

## Conversión de variables

Preprocesamiento: hay que convertir todos los features categóricos a numeros. Para variables categóricas se usan Dummy Variables y para ordinales OrdinalEncoder.

TODO: hay que hacer reducción dimensional antes, usando dummy variables se agranda la dimensión

### Educación alcanzada

In [11]:
from sklearn.preprocessing import OrdinalEncoder

categorias = [
 'preescolar',
 '1-4_grado',
 '5-6_grado',
 '7-8_grado',
 '9_grado',
 '1_anio',
 '2_anio',
 '3_anio',
 '4_anio',
 '5_anio',
 'universidad_1_anio',
 'universidad_2_anio',
 'universidad_3_anio',
 'universidad_4_anio',
 'universiada_5_anio',
 'universiada_6_anio'
]
oe = OrdinalEncoder(categories= [categorias])
X = df
X[["educacion_alcanzada_encoded"]] = oe.fit_transform(df[["educacion_alcanzada"]])
X = X.drop('educacion_alcanzada', axis=1)

df[["educacion_alcanzada", "educacion_alcanzada_encoded"]].drop_duplicates().sort_values(by="educacion_alcanzada_encoded")

,educacion_alcanzada,educacion_alcanzada_encoded
224,preescolar,0.0
160,1-4_grado,1.0
56,5-6_grado,2.0
15,7-8_grado,3.0
6,9_grado,4.0
77,1_anio,5.0
3,2_anio,6.0
415,3_anio,7.0
2,4_anio,8.0
10,5_anio,9.0


### Género

In [12]:
X = pd.get_dummies(X, columns=['genero'], dummy_na=False, drop_first=True)

### Estado marital

In [13]:
X = pd.get_dummies(X, columns=['estado_marital'], dummy_na=False, drop_first=True)

### Religión

In [14]:
X = pd.get_dummies(X, columns=['religion'], dummy_na=False, drop_first=True)

### Barrio

In [15]:
X = pd.get_dummies(X, columns=['barrio'], dummy_na=True, drop_first=True)

### Categoria de trabajo

In [16]:
X = pd.get_dummies(X, columns=['categoria_de_trabajo'], dummy_na=False, drop_first=True)

### Rol familiar

In [17]:
X = pd.get_dummies(X, columns=['rol_familiar_registrado'], dummy_na=False, drop_first=True)

### Trabajo

In [18]:
X = pd.get_dummies(X, columns=['trabajo'], dummy_na=False, drop_first=True)

## Selección de variables

(TODO: filtrar la variable de educación alcanzada por la alta correlación con los años estudiados)

¿Reducción dimensional?

# Árbol de decisión

Para entrenar el modelo:

In [19]:
from sklearn import preprocessing, tree

def get_tree(X, y, max_depth=5, min_samples_leaf=10):
    """Devuelve el árbol entrenado."""
    clf = tree.DecisionTreeClassifier(
        random_state=117, max_depth=max_depth, min_samples_leaf=min_samples_leaf
    )
    clf.fit(X, y)
    return clf

## Buscamos los hiperparámetros del árbol

Evitar overfitting (TODO: poda, criterio de parada)

In [ ]:
from sklearn.model_selection import GridSearchCV 
# GridSearch porque son solo 2 hiperparametros? preguntar, sino uso RandomSearch

## Métricas y cross validation


### Stratified K-folds
Dividimos el dataset en 5 folds para entrenar el modelo (estratificados para ser más homogéneos en cantidad de personas de altos ingresos):

In [39]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

X_array = np.array(X.drop('tiene_alto_valor_adquisitivo', axis=1))
y = np.array(X.tiene_alto_valor_adquisitivo)

kf = StratifiedKFold(n_splits=5)

max_depth = 11
min_samples_leaf = 4

test_accuracies = []
for train_index, test_index in kf.split(X, y):
    clf = get_tree(X_array[train_index], y[train_index], max_depth, min_samples_leaf)
    test_accuracy = accuracy_score(y[test_index], clf.predict(X_array[test_index]))
    test_accuracies.append(test_accuracy)

print(f"folds test accuracy is: {test_accuracies}")

folds test accuracy is: [0.8461538461538461, 0.8496621621621622, 0.8547297297297297, 0.8630221130221131, 0.8588759213759214]


### Curva ROC-AUC:

In [41]:
from sklearn.metrics import accuracy_score, roc_auc_score